## Analysis for Week 2: Mac Hand Data Project

In [2]:
import pandas as pd
import numpy as np

In [ ]:
#Importing data
data = pd.read_csv('./Data/MacHandData.csv')

,Subject id,dominance,affected side,age,sex,Task1,Total score
0,001H-L,0,0,52,0,5,97
1,001H-L,0,0,52,0,5,99
2,001H-R,0,0,52,0,4,82
3,001H-R,0,0,52,0,4,80
4,002H-L,1,1,47,0,4,58


### Summary statistics by individual

Gets number of trials for each participant, per hand.

In [ ]:
df = data.copy()

# Get individual id
df['individual id'] = df['Subject id'].str[:-2]

df2 = df.groupby(['individual id', 'Subject id']).size().reset_index(name='counts')
df2['hand'] = df2['Subject id'].str[-1]
df2 = df2.drop(columns=['Subject id'])


In [24]:
df2

,individual id,counts,hand
0,001H,2,L
1,001H,2,R
2,002H,2,L
3,002H,2,R
4,003H,3,L
...,...,...,...
79,040N,2,R
80,041N,3,L
81,041N,2,R
82,042N,3,L
